Задача 1: Выгрузка информации с использованием SQL-запроса. Начнем с анализа структуры базы данных.

Схема базы данных успешно загружена. На основе схемы я подготовлю SQL-запрос для получения данных, указанных в задании. Для этого потребуется информация из таблиц:

* courses: название курса, предмет, тип курса, дата старта.
* users: информация о учениках (ID, фамилия, город).
* course_users: связь между курсами и учениками, дата открытия курса ученику, статус отчисления.
* homework_done: количество сданных ДЗ ученика на курсе.

Сейчас составлю запрос. ​

In [ ]:
#SQL-запрос для выгрузки информации о пользователях
SELECT
    cu.course_id AS "ID курса",
    c.name AS "Название курса",
    s.name AS "Предмет",
    s.type AS "Тип предмета",
    ct.name AS "Тип курса",
    c.start_date AS "Дата старта курса",
    cu.user_id AS "ID ученика",
    u.last_name AS "Фамилия ученика",
    ci.name AS "Город ученика",
    cu.is_expelled AS "Ученик не отчислен с курса",
    cu.open_date AS "Дата открытия курса ученику",
    TIMESTAMPDIFF(MONTH, cu.open_date, CURRENT_DATE) AS "Сколько полных месяцев курса открыто у ученика",
    COUNT(hd.id) AS "Число сданных ДЗ ученика на курсе"
FROM
    course_users cu
INNER JOIN
    courses c ON cu.course_id = c.id
INNER JOIN
    subjects s ON c.subject_id = s.id
INNER JOIN
    course_types ct ON c.type_id = ct.id
INNER JOIN
    users u ON cu.user_id = u.id
INNER JOIN
    cities ci ON u.city_id = ci.id
LEFT JOIN
    homework_done hd ON cu.course_id = hd.course_id AND cu.user_id = hd.user_id
WHERE
    c.type_id IN (SELECT id FROM course_types WHERE name LIKE '%ЕГЭ%' OR name LIKE '%ОГЭ%')
    AND cu.is_expelled = 0
GROUP BY
    cu.course_id, cu.user_id;

Задача №-2: Потребуется Python-скрипт, чтобы обработать выгруженный датасет. Мы будем:

1.Проверять данные на:
* Дубликаты.
* Пропущенные значения.
* Соответствие типов данных.
* Аномальные значения.
2.Делить учеников на "волны" по дате их присоединения к курсу.


In [ ]:
# Create a Python script to analyze and process the dataset
python_script = """
import pandas as pd

# Загрузка данных из выгруженного CSV-файла
file_path = 'exported_data.csv'  # Укажите путь к вашему файлу
data = pd.read_csv(file_path)

# Проверка данных
print("Информация о датасете:")
print(data.info())

print("\\nПервые строки данных:")
print(data.head())

# Проверка на дубликаты
duplicates = data.duplicated()
print(f"Число дубликатов: {duplicates.sum()}")

# Удаление дубликатов, если они есть
if duplicates.sum() > 0:
    data = data.drop_duplicates()
    print("Дубликаты удалены.")

# Проверка на пропущенные значения
missing_values = data.isnull().sum()
print("\\nПропущенные значения:")
print(missing_values)

# Заполнение или удаление пропусков
# (Добавьте здесь логику обработки пропусков, если необходимо)

# Проверка на аномальные значения
# Например, логика проверки дат присоединения учеников:
print("\\nПроверка дат открытия курса ученикам...")
data['Дата открытия курса ученику'] = pd.to_datetime(data['Дата открытия курса ученику'], errors='coerce')
data['Дата старта курса'] = pd.to_datetime(data['Дата старта курса'], errors='coerce')

if data['Дата открытия курса ученику'].isnull().any():
    print("Найдены некорректные даты открытия курса.")

# Разделение учеников на волны
print("\\nДобавление волны...")
data['Волна'] = (data['Дата открытия курса ученику'] - data['Дата старта курса']).dt.days

def categorize_wave(days):
    if pd.isna(days) or days < 0:
        return None
    elif days <= 0:
        return '0 волна'
    elif days <= 7:
        return '1 волна'
    elif days <= 14:
        return '2 волна'
    elif days <= 21:
        return '3 волна'
    elif days <= 28:
        return '4 волна'
    else:
        return '5 волна'

data['Волна'] = data['Волна'].apply(categorize_wave)

# Сохранение обработанных данных в новый CSV-файл
output_path = 'processed_data.csv'
data.to_csv(output_path, index=False)
print(f"Обработанные данные сохранены в {output_path}")
"""

# Save the Python script to a file
script_file_path = "/mnt/data/Task2_DataProcessing.py"
with open(script_file_path, "w", encoding="utf-8") as file:
    file.write(python_script)

script_file_path

In [ ]:
import pandas as pd

# Загрузка данных из выгруженного CSV-файла
file_path = 'exported_data.csv'  # Укажите путь к вашему файлу
data = pd.read_csv(file_path)

# Проверка данных
print("Информация о датасете:")
print(data.info())

print("\nПервые строки данных:")
print(data.head())

# Проверка на дубликаты
duplicates = data.duplicated()
print(f"Число дубликатов: {duplicates.sum()}")

# Удаление дубликатов, если они есть
if duplicates.sum() > 0:
    data = data.drop_duplicates()
    print("Дубликаты удалены.")

# Проверка на пропущенные значения
missing_values = data.isnull().sum()
print("\nПропущенные значения:")
print(missing_values)

# Заполнение или удаление пропусков
# (Добавьте здесь логику обработки пропусков, если необходимо)

# Проверка на аномальные значения
# Например, логика проверки дат присоединения учеников:
print("\nПроверка дат открытия курса ученикам...")
data['Дата открытия курса ученику'] = pd.to_datetime(data['Дата открытия курса ученику'], errors='coerce')
data['Дата старта курса'] = pd.to_datetime(data['Дата старта курса'], errors='coerce')

if data['Дата открытия курса ученику'].isnull().any():
    print("Найдены некорректные даты открытия курса.")

# Разделение учеников на волны
print("\nДобавление волны...")
data['Волна'] = (data['Дата открытия курса ученику'] - data['Дата старта курса']).dt.days

def categorize_wave(days):
    if pd.isna(days) or days < 0:
        return None
    elif days <= 0:
        return '0 волна'
    elif days <= 7:
        return '1 волна'
    elif days <= 14:
        return '2 волна'
    elif days <= 21:
        return '3 волна'
    elif days <= 28:
        return '4 волна'
    else:
        return '5 волна'

data['Волна'] = data['Волна'].apply(categorize_wave)

# Сохранение обработанных данных в новый CSV-файл
output_path = 'processed_data.csv'
data.to_csv(output_path, index=False)
print(f"Обработанные данные сохранены в {output_path}")

Для Задачи 3 потребуется создать дашборд в Yandex DataLens, который будет:
1. Показывать сравнительный анализ курсов по числу и проценту продлений.

2. Давать возможность фильтровать учеников по ряду условий, включая волны, города и число выполненных домашних заданий.

Шаги выполнения:

1.Подготовка данных:

* Используем обработанный датасет, созданный на шаге 2.
* Убедиимся, что данные соответствуют требованиям задачи (например, поля с числом продлений и процентом продлений).

2.Настройка Yandex DataLens:

* Загрузите обработанный CSV-файл в DataLens как новый источник данных.
* Создайте необходимые вычисляемые поля:
* Число продлений 2-го месяца.
* Процент продлений 2-го месяца (число продлений деленное на общее количество учеников курса).

3.Создание визуализаций:

* Построим графики или таблицы для отображения сравнительных метрик курсов.
* Добавим фильтры по волнам, городам и другим критериям.

4.Публикация и предоставление ссылки:

* Настроим доступ к дашборду и создайте публичную ссылку.

Пример кода для вычисляемых показателей:

В DataLens вы можете создать такие формулы для расчетов:
* Число продлений 2-го месяца:

In [ ]:
COUNT(CASE WHEN "Месяц продления" = 2 THEN "ID ученика" END)

* Процент продлений 2-го месяца:

In [ ]:
COUNT(CASE WHEN "Месяц продления" = 2 THEN "ID ученика" END) * 100 / COUNT("ID ученика")